In [ ]:
import numpy as np

from dpipe.dataset.segmentation import SegmentationFromCSV
from dpipe.dataset.wrappers import Proxy
from dpipe.im.shape_ops import zoom
from dpipe.io import load
from liver_ct.liver_localization import SelectSlice
import os

class CC359(SegmentationFromCSV):
    def __init__(self, data_path, modalities= ['CT'], target='target', metadata_rpath='meta.csv', thresh=0.5):
        super().__init__(data_path=data_path,
                         modalities=modalities,
                         target=target,
                         metadata_rpath=metadata_rpath)
        self.df.index = range(len(self.df))
        self.threshold = thresh
    
    def get_indices(self, i):
        fname = os.path.join(self.path, self.df.loc[i].pred)
        print(fname)
        pred = load(fname)[0][0] > self.threshold
        print(pred.shape)
        ind_min, ind_max = SelectSlice(pred)
        return 2*ind_min, 2*ind_max
        
    def load_image(self, i):
        #fname = os.path.join(self.path, self.df.loc[i].CT)
        img = np.float32(super().load_image(i)[0])  # 4D -> 3D
        ind_min, ind_max = self.get_indices(i)
        return img[:,:,ind_min:ind_max]

    def load_segm(self, i):
        return   # already 3D
        img = np.float32(super().load_segm(i)==2)
        ind_min, ind_max = self.get_indices(i)
        return img[:,:,ind_min:ind_max]

    def load_shape(self, i):
        return np.int32(np.shape(self.load_segm(i)))


In [ ]:
dset = CC359("/home/egor/LITS/LITS_reoriented", metadata_rpath='meta_with_preds.csv')

In [ ]:
dset.path

In [ ]:
dset.df.head()

In [ ]:
dset.get_indices(5)

In [ ]:
seg = dset.load_segm(5)

In [ ]:
from dpipe.im.visualize import slice3d

In [ ]:
slice3d(seg)

In [ ]:
((seg[:,:, 378])==1).sum()